In [205]:
# Machine Learning Final
# Connor Hehn and John Minogue

import pandas as pd
import numpy as np

In [206]:
# Reading the data into data frames

df_1E = pd.read_csv("Data/Subject01E.csv")
df_1G = pd.read_csv("Data/Subject01G.csv")

df_2E = pd.read_csv("Data/Subject02E.csv")
df_2G = pd.read_csv("Data/Subject02G.csv")

df_3E = pd.read_csv("Data/Subject03E.csv")
df_3G = pd.read_csv("Data/Subject03G.csv")

df_4E = pd.read_csv("Data/Subject04E.csv")
df_4G = pd.read_csv("Data/Subject04G.csv")

df_5E = pd.read_csv("Data/Subject05E.csv")
df_5G = pd.read_csv("Data/Subject05G.csv")

df_6E = pd.read_csv("Data/Subject06E.csv")
df_6G = pd.read_csv("Data/Subject06G.csv")

df_7E = pd.read_csv("Data/Subject07E.csv")
df_7G = pd.read_csv("Data/Subject07G.csv")

df_8E = pd.read_csv("Data/Subject08E.csv")
df_8G = pd.read_csv("Data/Subject08G.csv")

df_9E = pd.read_csv("Data/Subject09E.csv")
df_9G = pd.read_csv("Data/Subject09G.csv")

df_10E = pd.read_csv("Data/Subject10E.csv")
df_10G = pd.read_csv("Data/Subject10G.csv")

In [207]:
# Creating

df_1E['Forefoot Force[N]'].describe()
df_new = df_1E[['Time[secs]','Forefoot Force[N]','Midfoot Force[N]','Heel Force[N]','TForce[N]','RAnkleFlexion','RAnkleInversion','RAnkleAbduction']].describe()

main_df = pd.DataFrame()



In [208]:
# Creating and merging dataframe

frame = [df_1E, df_1G, df_2E, df_2G, df_3E, df_3G, df_4E, df_4G, df_5E, df_5G, df_6E, df_6G, df_7E, df_7G, df_8E, df_8G, df_9E, df_9G, df_10E, df_10G]
main_dataframe = pd.concat(frame)

main_mean = (
    main_dataframe
    .groupby(['Person','Grounded'])[main_dataframe.columns]
    .mean()
)

main_mean = main_mean.add_suffix(" Avg")

main_min = main_dataframe.groupby(['Person','Grounded'])[main_dataframe.columns].min()
main_min = main_mean.add_suffix(" Avg")

main_max = main_dataframe.groupby(['Person','Grounded'])[main_dataframe.columns].max()
main_max = main_mean.add_suffix(" Avg")

main_mean




Grounded Avg  Time[secs] Avg  Forefoot Force[N] Avg  \
Person Grounded                                                        
1.0    0.0                0.0           2.505              268.17820   
       1.0                1.0           2.505              295.42568   
2.0    1.0                1.0           2.505              267.62222   
3.0    0.0                0.0           2.505              310.61624   
       1.0                1.0           2.505              244.95776   
4.0    0.0                0.0           2.505              323.95858   
       1.0                1.0           2.505              244.21504   
5.0    0.0                0.0           2.505              348.33180   
       1.0                1.0           2.505              347.59542   
6.0    0.0                0.0           2.505              345.04332   
       1.0                1.0           2.505              262.51362   
7.0    0.0                0.0           2.505              327.43248   
       1.0                1.0           2.505              262.91122   
8.0    0.0                0.0           2.505              332.62416   
       1.0                1.0           2.505              251.65320   
9.0    0.0                0.0           2.505              318.31566   
       1.0                1.0           2.505              262.91370   
10.0   0.0                0.0           2.505              288.56588   
       1.0                1.0           2.505              261.83958   

                 Midfoot Force[N] Avg  Heel Force[N] Avg  TForce[N] Avg  \
Person Grounded                                                           
1.0    0.0                  111.13470           37.17694      416.48984   
       1.0                   48.77620           33.17526      377.37714   
2.0    1.0                   53.72826           63.03920      384.38968   
3.0    0.0                   88.06160            8.91186      407.58970   
       1.0                   65.09824           79.34790      389.40390   
4.0    0.0                   51.23212            2.10990      377.30060   
       1.0                   61.20818           79.35768      384.78090   
5.0    0.0                   52.61484            2.42308      403.36972   
       1.0                   56.54548           74.26336      478.40426   
6.0    0.0                   38.00134           10.78602      393.83068   
       1.0                   66.66044           79.33126      408.50532   
7.0    0.0                   24.72770            2.83892      354.99910   
       1.0                   71.30802           75.20508      409.42432   
8.0    0.0                   49.20524            3.11394      384.94334   
       1.0                   75.20172           79.33878      406.19370   
9.0    0.0                   59.51156            3.77114      381.59836   
       1.0                   66.89666           74.22666      404.03702   
10.0   0.0                   51.80728            4.42070      344.79386   
       1.0                   70.75172           78.55714      411.14844   

                 RAnkleFlexion Avg  RAnkleInversion Avg  RAnkleAbduction Avg  \
Person Grounded                                                                
1.0    0.0               -6.116588             0.840514            10.655655   
       1.0              -17.955445             5.078729             9.191940   
2.0    1.0              -12.691091             3.295907             9.981309   
3.0    0.0               -7.305906             1.363744            10.066637   
       1.0              -12.266680             4.470045             9.323144   
4.0    0.0               -4.747487             2.850645            12.039727   
       1.0              -11.910789             3.374302            10.175655   
5.0    0.0               -4.668153             0.972078            12.483121   
       1.0              -14.486130             3.539233            10.238134   
6.0    0.0               -5.737951             0